In [1]:
from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy

# spacy is used to work on text
import spacy

#import other libraries
import numpy as np
import pandas as pd
import random
import sys
import os
import time
import codecs
import collections
import string
import re
from six.moves import cPickle

Using TensorFlow backend.


In [2]:
with open('word_list.txt', 'rb') as f:
    wordlist = cPickle.load(f)

In [3]:
wordlist = wordlist[:10000]

In [4]:
# count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

# #save the words and vocabulary
# with open('word_dict.txt', 'wb') as f:
#     cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  2712


In [5]:
rnn_size = 256 # size of RNN
seq_length = 30 # sequence length
learning_rate = 0.001 #learning rate
sequences_step = 1 #step to create sequences

In [6]:
#create sequences
sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 9970


In [25]:
# X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
# y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
# for i, sentence in enumerate(sequences):
#     for t, word in enumerate(sentence):
#         X[i, t, vocab[word]] = 1
#     y[i, vocab[next_words[i]]] = 1

In [26]:
# def bidirectional_lstm_model(seq_length, vocab_size):
#     print('Build LSTM model.')
#     model = Sequential()
#     model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
#     model.add(Dropout(0.6))
#     model.add(Dense(vocab_size))
#     model.add(Activation('softmax'))
    
#     optimizer = Adam(lr=learning_rate)
#     callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
#     print("model built!")
#     return model

In [28]:
# rnn_size = 256 # size of RNN
# seq_length = 30 # sequence length
# learning_rate = 0.001 #learning rate

# md = bidirectional_lstm_model(seq_length, vocab_size)
# md.summary()

Build LSTM model.
model built!
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 512)               6080512   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2712)              1391256   
_________________________________________________________________
activation_2 (Activation)    (None, 2712)              0         
Total params: 7,471,768
Trainable params: 7,471,768
Non-trainable params: 0
_________________________________________________________________


In [7]:
#load vocabulary
print("loading vocabulary...")
# vocab_file = os.path.join(save_dir, "words_vocab.pkl")

 
# with open('word_dict.txt', 'wb') as f:
#     cPickle.dump((words, vocab, vocabulary_inv), f)

# with open('word_dict.txt', 'rb') as f:
#     words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)

from keras.models import load_model
# load the model
print("loading model...")
model = load_model('my_model_generate_sentences.h5')

loading vocabulary...
loading model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
words_number = 30 # number of words to generate
seed_sentences = "he is walking past " #seed sentence to start the generating.

#initiate sentences
generated = ''
sentence = []

#we shate the seed accordingly to the neural netwrok needs:
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)

#the, we generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.33)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

#print the whole text
print(generated)

a a a a a a a a a a a a a a a a a a a a a a a a a a he is walking past and a rescuer fight on the jack and the another of the generation boone's as befriends an maiden as boone and his companion start out on a a a tree


In [10]:
generated[52:]

"he is walking past and a rescuer fight on the jack and the another of the generation boone's as befriends an maiden as boone and his companion start out on a a a tree"

In [11]:
words_number = 30 # number of words to generate
seed_sentences = "john would always go up the stairs only to see" #seed sentence to start the generating.

#initiate sentences
generated = ''
sentence = []

#we shate the seed accordingly to the neural netwrok needs:
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)

#the, we generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.33)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

#print the whole text
print(generated)

a a a a a a a a a a a a a a a a a a a a john would always go up the stairs only to see and fight in the – as the trusting after her brief after a indian still boone has a time to a indian some it has be be a has a


In [12]:
generated[40:]

'john would always go up the stairs only to see and fight in the – as the trusting after her brief after a indian still boone has a time to a indian some it has be be a has a'

In [ ]:
#char gen
#mini batch


# eval
# existing classifier
